# DAT405/DIT407 Introduction to Data Science and AI 
## 2022-2023, Reading Period 4
## Assignment 4: Spam classification using Naïve Bayes 
The exercise takes place in this notebook environment.
Hints:
You can execute certain linux shell commands by prefixing the command with `!`. You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results the second you can use writing code snippets that execute the tasks required.  

In this assignment you will implement a Naïve Bayes classifier in Python that will classify emails into spam and non-spam (“ham”) classes.  Your program should be able to train on a given set of spam and “ham” datasets. 
You will work with the datasets available at https://spamassassin.apache.org/old/publiccorpus/. There are three types of files in this location: 
-	easy-ham: non-spam messages typically quite easy to differentiate from spam messages. 
-	hard-ham: non-spam messages more difficult to differentiate 
-	spam: spam messages 

**Execute the cell below to download and extract the data into the environment of the notebook -- it will take a few seconds.** If you chose to use Jupyter notebooks you will have to run the commands in the cell below on your local computer, with Windows you can use 
7zip (https://www.7-zip.org/download.html) to decompress the data.

**What to submit:** 
Convert the notebook to a pdf-file and submit it. Make sure all cells are executed so all your code and its results are included. Double check the pdf displays correctly before you submit it.

In [ ]:
#Download and extract data
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2
!tar -xjf 20021010_easy_ham.tar.bz2
!tar -xjf 20021010_hard_ham.tar.bz2
!tar -xjf 20021010_spam.tar.bz2

*The* data is now in the three folders `easy_ham`, `hard_ham`, and `spam`.

In [ ]:
!ls -lah

### 1. Preprocessing: 

##### 1.1 Look at a few emails from easy_ham, hard_ham and spam. Do you think you would be able to classify the emails just by inspection? How do you think a succesful model can learn the difference between the different classes of emails?


In [72]:
# Write your code for here for looking a few emails
import os, random

ham = random.choice(os.listdir("./easy_ham/"))
spam = random.choice(os.listdir("./spam/"))

#print(fetch_format(f"./spam/{spam}"))
print(fetch_format(f"./easy_ham/{ham}"))

Nobel Honors 3 for Astrophysics Work 

Tuesday October 8, 2002 12:00 PM
STOCKHOLM, Sweden (AP) - Two Americans and a Japanese won the Nobel Prize in
physics Tuesday for using some of the most obscure particles and waves in
nature to understand the workings of astronomy's grandest wonders.
Riccardo Giacconi, 71, of the Associated Universities Inc. in Washington,
D.C., will get half of the $1 million prize for his role in ``pioneering
contributions to astrophysics, which have led to the discovery of cosmic
X-ray sources.''
Raymond Davis, Jr., 87, of the University of Pennsylvania shares the other
half of the prize with Japanese scientist Masatoshi Koshiba, 76, of the
University of Tokyo. The two men pioneered the construction of giant
underground chambers to detect neutrinos, elusive particles that stream from
the sun by the billion.
Neutrinos offer an unparalleled view of the sun's inner workings because
they are produced in its heart by the same process that causes it to shine.
In fact

Answer 1.1:
It seems feasible to classify the emails only from inspection. It seems like a lot of spam mails contains words like "invest", "click" and "100%" etcetera. To create a model for this we could generate some statistics on which words often appear in spam mails but seldom in ham mails. Then we would know which words to look for. Looking through a mail we could generate a score based on how many of these words appear and taking into account how statistically probable they are to be in a spam mail. Another thing to look for is HTML tags. It seems a lot of spam mails contains HTML whereas ham mails doesn't. Simply looking for HTML-tags would suffice for this since that is needed for any HTML to work. 

##### 1.2 Note that the email files contain a lot of extra information, besides the actual message. Ignore that for now and run on the entire text (in the optional part further down can experiment with filtering out the headers and footers). We don’t want to train and test on the same data (it might help to reflect on why if you don't recall). Split the spam and the ham datasets in a training set and a test set. (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`). Use only the easy_ham part as ham data for quesions 1 and 2.

In [74]:
# Write your code for here for splitting the data
from sklearn.model_selection import train_test_split

ham_paths = os.listdir("./easy_ham/")
spam_paths = os.listdir("./spam/")

hamtrain_paths, hamtest_paths = train_test_split(ham_paths, test_size=0.3)
spamtrain_paths, spamtest_paths = train_test_split(spam_paths, test_size=0.3)

hamtrain = [fetch_format(f"./easy_ham/{x}") for x in hamtrain_paths]
hamtest = [fetch_format(f"./easy_ham/{x}") for x in hamtest_paths]
spamtrain = [fetch_format(f"./spam/{x}") for x in spamtrain_paths]
spamtest = [fetch_format(f"./spam/{x}") for x in spamtest_paths]

### 2.1 Write a Python program that: 
1.	Uses the four datasets from Question 1 (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`) 
2.	Trains a Naïve Bayes classifier (use the [scikit-learn library](https://scikit-learn.org/stable/)) on `hamtrain` and `spamtrain`, that classifies the test sets and reports True Positive and False Negative rates on the `hamtest` and `spamtest` datasets. Use `CountVectorizer` ([Documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)) to transform the email texts into vectors. Please note that there are different types of Naïve Bayes Classifier in scikit-learn ([Documentation here](https://scikit-learn.org/stable/modules/naive_bayes.html)). Test two of these classifiers that are well suited for this problem:
- Multinomial Naive Bayes  
- Bernoulli Naive Bayes. 

Please inspect the documentation to ensure input to the classifiers is appropriate before you start coding. 



In [77]:
# Write your code here
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer

gnb = GaussianNB()

ham_vectorizer = CountVectorizer()
ham_vectorizer.fit(hamtrain)
spam_vectorizer = CountVectorizer()
spam_vectorizer.fit(spamtrain)

htr_vector = ham_vectorizer.transform(hamtrain)
str_vector = spam_vectorizer.transform(spamtrain)

gnb.fit(htr_vector.toarray(), str_vector.toarray().reshape(-1,1))
y_pred = gnb.predict(hamtest)

/home/hannes/Plugg/DAT405/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: Found input variables with inconsistent numbers of samples: [1785, 12511450]

### 2.2 Answer the following questions:
##### a) What does the CountVectorizer do?
Answer 2.2.a
##### b) What is the difference between Multinomial Naive Bayes and Bernoulli Naive Bayes
Answer 2.2.b


### 3.1 Run the two models:
Run (don't retrain) the two models from Question 2 on spam versus hard-ham. Does the performance differ compared to question 2 when the model was run on spam versus easy-ham? If so, why?

In [ ]:
# Write your code here

Answer 3.1:

### 3.2 Retrain
Retrain new Multinomial and Bernolli Naive Bayes classifers on the combined (easy+hard) ham and spam. Now evaluate on spam versus hard-ham as in 3.1. Also evaluate on spam versus easy-ham. Compare the performance with question 2 and 3.1. What do you observe?

In [ ]:
# Write your code here

Answer 3.2:

### 3.3 Further improvements
Do you have any suggestions for how performance could be further improved? You don't have to implement them, just present your ideas.

Answer 3.3:

In [48]:
# Fetching text from a file path and removing the header info
def fetch_format(path):
   with open(path, 'r', encoding='latin1') as f:
      split = f.read().split('\n\n', 1)
   if len(split) == 1:
      return split[0]
   return split[1]